In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### requiere la carpeta de cora de https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz
### cora.cites es un grafo que une a dos paper_id cuando uno cita al otro


## Dataset de CORA

Este dataset contiene dos archivos. El primero es una base de datos de papers que citan a otros papers

In [2]:
citas = pd.read_csv('cora/cora.cites',sep="\t",
    header=None,
    names=["target", "source"])

citas

,target,source
0,35,1033
1,35,103482
2,35,103515
3,35,1050679
4,35,1103960
...,...,...
5424,853116,19621
5425,853116,853155
5426,853118,1140289
5427,853155,853118


El segundo archivo contiene informacién de muchas palabras (términos) asociadas a cada paper, y además una categoría. 

In [3]:
###informacion de los papers de acuerdo a loas palabras que mencionan, junto a su paper id y el tema general

column_names = ["paper_id"] + [f"word_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    'cora/cora.content', sep="\t", names=column_names,
)
papers

,paper_id,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,subject
0,31336,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Neural_Networks
1,1061127,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,Rule_Learning
2,1106406,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
3,13195,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Reinforcement_Learning
4,37879,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Probabilistic_Methods
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,1128975,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2704,1128977,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2705,1128978,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Genetic_Algorithms
2706,117328,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Case_Based


In [4]:
print(papers.subject.value_counts())


Neural_Networks           818
Probabilistic_Methods     426
Genetic_Algorithms        418
Theory                    351
Case_Based                298
Reinforcement_Learning    217
Rule_Learning             180
Name: subject, dtype: int64


Vamos a pasar estos subjects a números, y armar los paper_id para que sean consecutivos

In [5]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citas["source"] = citas["source"].apply(lambda name: paper_idx[name])
citas["target"] = citas["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

In [6]:
citas

,target,source
0,0,21
1,0,905
2,0,906
3,0,1909
4,0,1940
...,...,...
5424,1873,328
5425,1873,1876
5426,1874,2586
5427,1876,1874


In [7]:
print(papers.subject.value_counts())

2    818
3    426
1    418
6    351
0    298
4    217
5    180
Name: subject, dtype: int64


### Sets de training y test
Lo último antes de empezar: dividir en dos este dataset (y tratar que nos queden bien balanceados los subjects)

In [33]:
#Primero sacamos un 50% de nodos de cada subject
train_data, test_data = [], []

for nombres, datos_agrupados in papers.groupby("subject"):
    random_selection = np.random.rand(len(datos_agrupados.index)) <= 0.5
    train_data.append(datos_agrupados[random_selection])
    test_data.append(datos_agrupados[~random_selection])
train_data = pd.concat(train_data)
test_data = pd.concat(test_data)

In [9]:
train_data

,paper_id,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,...,word_1424,word_1425,word_1426,word_1427,word_1428,word_1429,word_1430,word_1431,word_1432,subject
13,1233,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,771,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32,888,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,2658,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,1237,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2675,593,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2685,900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2689,1019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2694,1857,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,6


Nuestra tarea consiste en predecir la línea "subject" (el tema del paper), basándonos por ahora solo en el vector de ocurrencias de palabras. Con esto, dividimos el set de training y test en una matriz x y un vector y con los subjects. 

In [34]:
# Y luego algunas cosas que se necesitan    
feature_names = set(papers.columns) - {"paper_id", "subject"}
num_features = len(feature_names)
num_classes = len(class_idx)

# A keras le gustan los arreglos numpy
x_train = train_data[feature_names].to_numpy()
x_test = test_data[feature_names].to_numpy()

y_train = train_data["subject"]
y_test = test_data["subject"]

In [35]:
x_train, y_train

(array([[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 13      0
 32      0
 47      0
 50      0
 66      0
        ..
 2653    6
 2690    6
 2694    6
 2695    6
 2696    6
 Name: subject, Length: 1330, dtype: int64)

## La clase modelo en Keras

Para definir capas en una GNN tenemos que extender la clase *model* en Keras. Para orientarnos con esta funcionalidad vamos a crear primero una red neuronal básica que nos permita comparar resultados. La idea es que esta red básica sirva para predecir los temas de los papers usando solamente los features de cada paper (nodo en el grafo), y no sus conexiones. 


### Un perceptron multicapa (multilayered perceptron, MLP)
Esta primera función nos va a retornar un objeto Keras. Recibe un arreglo *capas_internas* de una dimension ([32,32]) y entrega un MLP completamente conectado, con capas de esas dimensiones, y con operaciones usuales (normalizaciòn, dropout, activaciòn no lineal) en cada capa. 

In [12]:
def create_MLP(capas_internas, dropout_rate, name=None):
    capas = []

    for capa_interna in capas_internas:
        capas.append(layers.BatchNormalization())
        capas.append(layers.Dropout(dropout_rate))
        capas.append(layers.Dense(capa_interna, activation=tf.nn.gelu))

    return keras.Sequential(capas, name=name)


### un primer clasificador. 

Recordemos que la tarea es predecir el subject de un paper, basado en el vector de ocurrencias de palabras. 
Nuestro clasificador tiene una primera capa para pasar de las dimensiones del input a la dimension de *capas_internas*, tres MLP conectados y una última capa para hacer la predicción. Los MLP están conectados secuencialmente, pero a su vez establecemos unas "skip connections" en el grafo de cómputo final. Tensorflow se encarga por detrás de hacer que todo propague bien para el entrenamiento. 

In [25]:
class BaseClassifier(tf.keras.Model):
    def __init__(
        self,
        num_classes,
        capas_internas = [32,32],
        dropout_rate=0.5,
        normalize=True,
        *args,
        **kwargs,
    ):
        super(BaseClassifier, self).__init__(*args, **kwargs)

        
        ### Definimos las capas de nuestro modelo como atributos. 
        ## Una capa que preprocese node_features y los lleve a una dimensionalidad mas pequeña
        
        self.preprocesador = create_MLP(capas_internas, dropout_rate, name="preprocesador")

        ## Tres capas adicionales 
        
        self.capa1 = create_MLP(capas_internas, dropout_rate, name="capa1")
        self.capa2 = create_MLP(capas_internas, dropout_rate, name="capa2")
        self.capa3 = create_MLP(capas_internas, dropout_rate, name="capa3")

        ## La ultima capa que llegue a num_clases 

        #self.clasificador = layers.Dense(units=num_classes, name="clasificador", activation=tf.keras.activations.softmax)
        self.clasificador = layers.Dense(units=num_classes, name="clasificador")
        
    ### La función call es la que se llama cuando se entrena o cuando se quiere predecir con una red ya entrenada. 
    ### Aquí es donde tenemos que explicarle a tensorflow como hacer las operaciones. 
    
    def call(self,input_features):
            
        i = self.preprocesador(input_features)
        c1 = self.capa1(i)
            
            #skip connection
        s1 = c1 + i 
            
            #otra capa, skip, capa, skip
        c2 = self.capa2(s1)
        s2 = c2 + s1 
        c3 = self.capa3(s2)
        s3 = c3 + s2
        

        return self.clasificador(s3)
        
        
        

### Entrenando el modelo

In [36]:
#Creamos un objeto BaseClassifier, tenemos 7 categorías de papers. 

BASE = BaseClassifier(
    num_classes=7,
    name="modelo_base",
)

In [37]:
#Compilamos el clasificador. Establecemos la learning rate, el loss y la métrica. Esto es estandar para un 
#clasificador discreto
BASE.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

In [38]:
# Early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_acc", patience=50, restore_best_weights=True
)

#Y finalmente ajustamos
history = BASE.fit(
    x=x_train,
    y=y_train,
    epochs=300,
    batch_size=256,
    validation_split=0.15,
    callbacks=[early_stopping],
)

Epoch 1/300
5/5 [==============================] - 3s 84ms/step - loss: 2.7839 - acc: 0.2363 - val_loss: 2.2707 - val_acc: 0.0000e+00
Epoch 2/300
5/5 [==============================] - 0s 15ms/step - loss: 2.1823 - acc: 0.2584 - val_loss: 2.4093 - val_acc: 0.0000e+00
Epoch 3/300
5/5 [==============================] - 0s 14ms/step - loss: 1.8488 - acc: 0.3575 - val_loss: 2.4843 - val_acc: 0.0000e+00
Epoch 4/300
5/5 [==============================] - 0s 14ms/step - loss: 1.8102 - acc: 0.3097 - val_loss: 2.5408 - val_acc: 0.0000e+00
Epoch 5/300
5/5 [==============================] - 0s 16ms/step - loss: 1.6157 - acc: 0.3699 - val_loss: 2.6171 - val_acc: 0.0000e+00
Epoch 6/300
5/5 [==============================] - 0s 15ms/step - loss: 1.5547 - acc: 0.4124 - val_loss: 2.6721 - val_acc: 0.0000e+00
Epoch 7/300
5/5 [==============================] - 0s 14ms/step - loss: 1.4558 - acc: 0.4434 - val_loss: 2.6834 - val_acc: 0.0000e+00
Epoch 8/300
5/5 [==============================] - 0s 16ms/ste

5/5 [==============================] - 0s 16ms/step - loss: 0.3590 - acc: 0.8708 - val_loss: 6.6558 - val_acc: 0.0800
Epoch 64/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3470 - acc: 0.8796 - val_loss: 6.7914 - val_acc: 0.0700
Epoch 65/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3541 - acc: 0.8788 - val_loss: 6.9283 - val_acc: 0.0650
Epoch 66/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3308 - acc: 0.8885 - val_loss: 6.8420 - val_acc: 0.0800
Epoch 67/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3602 - acc: 0.8726 - val_loss: 6.9679 - val_acc: 0.0750
Epoch 68/300
5/5 [==============================] - 0s 17ms/step - loss: 0.4045 - acc: 0.8558 - val_loss: 7.2420 - val_acc: 0.0700
Epoch 69/300
5/5 [==============================] - 0s 14ms/step - loss: 0.2820 - acc: 0.8965 - val_loss: 7.4991 - val_acc: 0.0700
Epoch 70/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3302 - acc: 0.8752 - 

5/5 [==============================] - 0s 15ms/step - loss: 0.3037 - acc: 0.8796 - val_loss: 8.8480 - val_acc: 0.0850
Epoch 126/300
5/5 [==============================] - 0s 15ms/step - loss: 0.2711 - acc: 0.8982 - val_loss: 9.0701 - val_acc: 0.0950
Epoch 127/300
5/5 [==============================] - 0s 14ms/step - loss: 0.2829 - acc: 0.9106 - val_loss: 9.1269 - val_acc: 0.0900
Epoch 128/300
5/5 [==============================] - 0s 14ms/step - loss: 0.2715 - acc: 0.9080 - val_loss: 9.0075 - val_acc: 0.0900
Epoch 129/300
5/5 [==============================] - 0s 13ms/step - loss: 0.2769 - acc: 0.8929 - val_loss: 9.1129 - val_acc: 0.0850
Epoch 130/300
5/5 [==============================] - 0s 14ms/step - loss: 0.3029 - acc: 0.8894 - val_loss: 8.8925 - val_acc: 0.0900
Epoch 131/300
5/5 [==============================] - 0s 14ms/step - loss: 0.2882 - acc: 0.8832 - val_loss: 8.9885 - val_acc: 0.0900
Epoch 132/300
5/5 [==============================] - 0s 15ms/step - loss: 0.3237 - acc: 0.

In [31]:
BASE.summary()

Model: "modelo_base"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 preprocesador (Sequential)  (None, 32)                52804     
                                                                 
 capa1 (Sequential)          (None, 32)                2368      
                                                                 
 capa2 (Sequential)          (None, 32)                2368      
                                                                 
 capa3 (Sequential)          (None, 32)                2368      
                                                                 
 clasificador (Dense)        multiple                  231       
                                                                 
Total params: 60,139
Trainable params: 56,825
Non-trainable params: 3,314
_________________________________________________________________


In [39]:
BASE.evaluate(x=x_test, y=y_test)

44/44 [==============================] - 0s 1ms/step - loss: 2.0741 - acc: 0.6611


[2.0741066932678223, 0.6611030697822571]

Malazo el modelo este 